In [1]:
from datetime import datetime
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.python.ops.gradients_impl import _hessian_vector_product

In [2]:
class_count = 2

def generate_two_classes_data(feature_count: int, object_count: int):
    """
    Generates two classes: the first class has features near 0, the second – near 1.
    :returns features (with bias=1 feature), answers
    """
    features = np.concatenate((
        np.random.normal(
            np.concatenate((
                np.zeros([object_count // 2, feature_count], dtype=np.float64),
                np.ones([object_count - object_count // 2, feature_count], dtype=np.float64)
            )),
            scale=0.1
        ),
        np.ones([object_count, 1], dtype=np.float64)
    ), axis=1)

    answers = np.concatenate((
        np.concatenate((
            np.zeros([object_count // 2, 1], dtype=np.float64),
            np.ones([object_count - object_count // 2, 1], dtype=np.float64)
        )),
        np.concatenate((
            np.ones([object_count // 2, 1], dtype=np.float64),
            np.zeros([object_count - object_count // 2, 1], dtype=np.float64)
        ))
    ), axis=1)

    return features, answers

f, a = generate_two_classes_data(2, 3)
print(f)
print(a)

[[-0.07523093  0.24884787  1.        ]
 [ 1.06208307  1.16458903  1.        ]
 [ 1.06368328  0.85945906  1.        ]]
[[0. 1.]
 [1. 0.]
 [1. 0.]]


In [9]:
np.random.seed(0)

times = 10

stats = pd.DataFrame(columns=['Feature Count', 'Gradient Avg Time (secs)', 'H-Vec Product Avg Time (secs)'])

# for feature_count in (2,):
# for feature_count in (10**2, 10**3, 10**4):
for feature_count in (10**2, 10**3, 10**4, 10**5, 10**6):
    object_count = 200

    features, answers = generate_two_classes_data(feature_count=feature_count, object_count=object_count)

    x = tf.placeholder(tf.float64, [None, feature_count + 1])
    y = tf.placeholder(tf.float64, [None, class_count])

    W = tf.Variable(tf.ones([feature_count + 1, class_count], tf.float64))

    predictions = tf.nn.softmax(tf.matmul(x, W))
    cost = tf.nn.sigmoid_cross_entropy_with_logits(logits=predictions, labels=y)

    g = tf.gradients(cost, W)[0]
    h_vec_product = _hessian_vector_product(cost, [W], [W])[0]

    init = tf.global_variables_initializer()

    g_deltas = []
    h_vec_deltas = []

    for i in range(times):
        with tf.Session() as sess:
            sess.run(init)

            start = datetime.now().timestamp()
            sess.run(g, feed_dict={x: features, y: answers})
            g_deltas.append(datetime.now().timestamp() - start)

            start = datetime.now().timestamp()
            sess.run(h_vec_product, feed_dict={x: features, y: answers})
            h_vec_deltas.append(datetime.now().timestamp() - start)

    g_delta = sum(g_deltas) / len(g_deltas)
    h_vec_delta = sum(h_vec_deltas) / len(h_vec_deltas)
    print(f"Features: {feature_count}, g delta: {g_delta}, h-vec delta: {h_vec_delta}")
    stats.loc[len(stats)] = [feature_count, g_delta, h_vec_delta]

Features: 100, g delta: 0.06799955368041992, h-vec delta: 0.11040019989013672
Features: 1000, g delta: 0.07449970245361329, h-vec delta: 0.1128000020980835
Features: 10000, g delta: 0.08189992904663086, h-vec delta: 0.11960046291351319
Features: 100000, g delta: 0.11875019073486329, h-vec delta: 0.15729970932006837
Features: 1000000, g delta: 0.4411057472229004, h-vec delta: 0.5117329597473145


In [10]:
stats

,Feature Count,Gradient Avg Time (secs),H-Vec Product Avg Time (secs)
0,100.0,0.068000,0.110400
1,1000.0,0.074500,0.112800
2,10000.0,0.081900,0.119600
3,100000.0,0.118750,0.157300
4,1000000.0,0.441106,0.511733
